In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
from urllib.request import urlopen
import json

In [3]:
# Get shapes of counties
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# Get FIPS to county name data 
url = 'https://raw.githubusercontent.com/ChuckConnell/articles/master/fips2county.tsv'
fips_and_counties = pd.read_csv(url, dtype='str', sep='\t', 
                                usecols=['StateName', 'CountyName', 'CountyFIPS', 'STATE_COUNTY'])

# Fix ñ character
fips_and_counties.iloc[1802] = ['Dona Ana', 'New Mexico', '35013', 'NM | DONA ANA']

for col in ['CountyName', 'StateName']:
    fips_and_counties[col] = fips_and_counties[col].apply(lambda x: x.lower())

In [4]:
missing_counties = [('alaska', 'petersburg census area', '02195', 'AK | PETERSBURG CENSUS AREA'),
('alaska', 'wade hampton census area', '02270', 'AK | WADE HAMPTON CENSUS AREA'),
('louisiana', 'la salle parish', '22059', 'LA | LA SALLE PARISH'),
('south dakota', 'shannon', '46113', 'SD | SHANNON'),
('virginia', 'bedford city', '51515', 'VA | BEDFORD CITY')]

missing_counties = pd.DataFrame(missing_counties,
                                columns=['StateName', 'CountyName', 'CountyFIPS', 'STATE_COUNTY'])

fips_and_counties = pd.concat([fips_and_counties, missing_counties], ignore_index=True)

In [5]:
fips_to_counties = fips_and_counties.set_index('CountyFIPS')['STATE_COUNTY'].to_dict()

def format_county(name):
    # DC is special
    if name == 'DC | DISTRICT OF COLUMBIA':
        return 'District Of Columbia | DC'
    
    # Title case the county name
    title_case = name[5:].title()
    last_word = name.split(' ')[-1]
    
    # Check if the word County should be added to the end
    if last_word not in ['AREA', 'BOROUGH', 'PARISH', 'CITY', 'MUNICIPALITY']:
        title_case += ' County'
        
    title_case = f'{title_case} | {name[:2]}'
    
    return title_case 

fips_to_counties = {key: format_county(val) for key, val in fips_to_counties.items()}

In [6]:
location_fips_dict = fips_and_counties.set_index(['StateName', 'CountyName'])['CountyFIPS'].to_dict()

def state_location_to_fips(row):
    state = row['State']
    
    location = row['Location'].split(' ')
    
    loc_name = ' '.join(location[:-1]) if location[-1]=='County' else row['Location']
    
    key = (state.lower(), loc_name.lower())
    
    if key not in location_fips_dict:
        print(key)
        return 'ERROR'
    
    return location_fips_dict[key]

# School Finance Data

In [24]:
# Data from https://nces.ed.gov/ccd/files.asp#Fiscal:1,LevelId:5,Page:1
# Documentation at https://nces.ed.gov/ccd/pdf/2020309_FY18F33_Documentation.pdf
school_finance = pd.read_csv('./Raw Data/sdf19_1a.txt', sep='\t', dtype={'CONUM':'str'},
                            usecols=['CONUM', 'MEMBERSCH', 'E13', 'TOTALEXP'])

# Group school finance data by county
county_expenditure = school_finance.groupby(['CONUM'])[['MEMBERSCH', 'E13']].sum()
county_expenditure['FIPS'] = county_expenditure.index

# Calculate Instruction Per Student
county_expenditure['Instruction Spending Per Student, 2018'] = county_expenditure['E13']/county_expenditure['MEMBERSCH']

# Drop erroneous data
county_expenditure = county_expenditure[county_expenditure['Instruction Spending Per Student, 2018']>0]

# Add place names
county_expenditure['PlaceName'] = county_expenditure['FIPS'].apply(lambda x: fips_to_counties[x])

county_expenditure.to_csv('./Cleaned Data/expenditure.csv',index=False)

# Diabetes Data

In [18]:
# Data from https://gis.cdc.gov/grasp/diabetes/DiabetesAtlas.html#
diabetes_data = pd.read_csv('./Raw Data/DiabetesAtlasCountyData.csv', dtype={'CountyFIPS':'str'},
                           usecols=['Percentage', 'CountyFIPS'])

# Clean diabetes data
diabetes_data = diabetes_data[diabetes_data['Percentage']!='No Data']
diabetes_data['Percentage'] = diabetes_data['Percentage'].astype('f8')
diabetes_data = diabetes_data.rename({'CountyFIPS':'FIPS', 
                                      'Percentage':'Diabetes Percentage, 2019'}, axis=1)

# Add Place Names
diabetes_data['PlaceName'] = diabetes_data['FIPS'].apply(lambda x: fips_to_counties[x])

diabetes_data.to_csv('./Cleaned Data/diabetes.csv',index=False)

# Alcohol Consumption Data

In [29]:
# Data from https://ghdx.healthdata.org/us-data
alcohol_data = pd.read_excel('./Raw Data/IHME_county_data_Alcohol_NATIONAL.xlsx', sheet_name=2,
                            usecols=['State', 'Location', '2012 Both Sexes'])

alcohol_data = alcohol_data.rename({'2012 Both Sexes':'Heavy Drinking Percentage, 2012'}, axis=1)

alcohol_data = alcohol_data[1:]
alcohol_data = alcohol_data[alcohol_data['Location'] != alcohol_data['State']]

alcohol_data['Location'] = alcohol_data['Location'].apply(lambda x: x.split(', '))
alcohol_data = alcohol_data.explode('Location')

alcohol_data['FIPS'] = alcohol_data.apply(state_location_to_fips, axis=1)

# Add Place Names
alcohol_data['PlaceName'] = alcohol_data['FIPS'].apply(lambda x: fips_to_counties[x])

alcohol_data.to_csv('./Cleaned Data/alcohol.csv',index=False)

# Life Expectancy

In [20]:
# Data from https://ghdx.healthdata.org/us-data
life_data = pd.read_excel('./Raw Data/IHME_USA_COUNTY_LE_MORTALITY_RISK_1980_2014_NATIONAL_Y2017M05D08.XLSX',
                          sheet_name=0, usecols=['FIPS', 'Life expectancy, 2014*'], dtype={'FIPS':'str'})

life_data = life_data[life_data['FIPS'].apply(len)>3]
life_data['FIPS'] = life_data['FIPS'].apply(lambda x: '0'+x if len(x)<5 else x)
life_data['PlaceName'] = life_data['FIPS'].apply(lambda x: fips_to_counties[x])


life_data['Life expectancy, 2014*'] = life_data['Life expectancy, 2014*'].apply(lambda x: float(x[:5]))
life_data = life_data.rename({'Life expectancy, 2014*':'Life expectancy, 2014'}, axis=1)

life_data.to_csv('./Cleaned Data/expectancy.csv',index=False)

# Visualizations

In [16]:
def make_map(df, variable, title = None, quant=0.01):
    crange = tuple(df[variable].quantile([quant,1-quant]))
    map_fig = px.choropleth(df, geojson=counties, locations='FIPS', color=variable,
                            color_continuous_scale="Viridis", range_color=crange,
                            scope="usa", hover_name = 'PlaceName',
                            hover_data = {'FIPS':False}, title=title)
    
    map_fig.update_coloraxes(colorbar={'orientation':'h', 'thickness':10, 'y':-0.1, 'len':0.7,
                                      'title':{'text':''}})
    
    map_fig.update_layout(margin=dict(l=0, r=0, t=20, b=20))
    
    return map_fig

def merged_scatter(dfs, variables, title=None):
    df = pd.merge(left=dfs[0], right=dfs[1], left_on='FIPS', right_on='FIPS')
    
    fig = px.scatter(df, x=variables[0], y=variables[1], trendline="ols", hover_name='PlaceName_x',
                     trendline_color_override="red",
                     marginal_x='rug', marginal_y='rug')
    
    results = px.get_trendline_results(fig).iloc[0][0]
    
    return fig, results.summary().as_text()

In [12]:
# Restrict to main body of schools
limited_exp = county_expenditure[county_expenditure['InstPerStudent']<15000]

In [13]:
fig, results = merged_scatter((limited_exp, diabetes_data), ['InstPerStudent', 'Percentage'])

fig, results = merged_scatter((limited_exp, alcohol_data), ['InstPerStudent', '2012 Both Sexes'])

fig.show()

results.summary()

fig, results = merged_scatter((limited_exp, life_data), ['InstPerStudent', 'Life expectancy, 2014*'], logit=False)

fig.show()

results.summary()

# Dash ??

In [30]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [28]:
scatter_data = {
    'Logistic Diabetes': (diabetes_data, 'Logit Percentage'), 
    'Logistic Alcohol': (alcohol_data, 'Logit 2012 Both Sexes'),
    'Life': (life_data, 'Life expectancy, 2014*')
}

scatter_sources = {key: merged_scatter((limited_exp, val[0]), ['InstPerStudent', val[1]])
                   for key, val in scatter_data.items()}

map_data = {
    'Per Student Instructional Spending': (county_expenditure, 'InstPerStudent'),
    'Percentage of Diabetes': (diabetes_data, 'Percentage'),
    'Percentage of Heavy Drinking': (alcohol_data, '2012 Both Sexes'),
    'Life Expectancy': (life_data, 'Life expectancy, 2014*')
}

map_sources = {key: make_map(val[0], val[1]) for key, val in map_data.items()}

In [32]:
# Build App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.Div([
        html.H1("Compare Two Maps of the US", 
                style={'display':'flex', 'justify-content': 'center'}),
        html.Div([dcc.Graph(id='map-1'), dcc.Graph(id='map-2')], 
                 style={'display':'flex', 'flex-direction':'row'}),
        html.Div([
            html.Label([
                'Variable 1',
                dcc.Dropdown(
                    id='map-1-dropdown', clearable=False, value='Per Student Instructional Spending', 
                    options=[{'label': c, 'value': c} for c in map_sources.keys()]
                )
            ], style={'width': '20%', 'display': 'inline-block'}),
            
            html.Label([
                'Variable 2',
                dcc.Dropdown(
                    id='map-2-dropdown', clearable=False, value='Per Student Instructional Spending', 
                    options=[{'label': c, 'value': c} for c in map_sources.keys()]
                )
            ], style={'width': '20%', 'display': 'inline-block'}),
        ], style={'display':'flex', 'justify-content':'space-around'}),
    ]),
    html.H1("The Impact of Per Student Instructional Spending"),
    dcc.Graph(id='scatter'),
    html.Label([
        "Data Source",
        dcc.Dropdown(
            id='scatter-dropdown', clearable=False,
            value='Logistic Diabetes', options=[
                {'label': c, 'value': c}
                for c in scatter_sources.keys()
            ])
    ]),
    html.Div(id='summary', 
             style={"white-space": "pre", 'font-family': 'Monaco'})
],  
    style={'font-family':'Roboto',
          'background-color':'#EEE'})
# Define callback to update graph
@app.callback(
    [Output('scatter', 'figure'),
     Output('summary', 'children')],
    [Input("scatter-dropdown", "value")]
)
def update_scatter(value):
    return scatter_sources[value]

@app.callback(
    Output('map-1', 'figure'),
    [Input("map-1-dropdown", "value")]
)
def update_map(value):
    return map_sources[value]

@app.callback(
    Output('map-2', 'figure'),
    [Input("map-2-dropdown", "value")]
)
def update_map(value):
    return map_sources[value]


app.run_server('external')

Dash app running on http://127.0.0.1:8050/
